##**Домашнее задание по теме "Коллаборативная фильтрация"**

**Задание:**

**Преподаватель:** Наталья Баданина, Иван Анисковец, Юлия Пономарева, Ярослав Сапронов, Егор Шишковец, Павел Мехнин

Пакет SURPRISE:

* используйте данные MovieLens 1M,
* можно использовать любые модели из пакета,
* получите RMSE на тестовом сете 0,87 и ниже.

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

**Решение:**

## Данные в MovieLens

1. **movies**:
- **Описание**: Содержит информацию о фильмах.
- `movieId`: Уникальный идентификатор фильма.
- `title`: Название фильма.
- `genres`: Жанры фильма, обычно представлены в виде строки со списком жанров, разделенных символами `|` (например, "Action|Comedy").

2. **ratings**:
- **Описание**: Содержит оценки фильмов, выставленные пользователями.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `rating`: Оценка (обычно от 0.5 до 5, с шагом 0.5).
- `timestamp`: Временная метка, указывающая, когда была оставлена оценка (обычно в формате Unix).


3. **tags** (не всегда присутствует):
- **Описание**: Содержит метки, оставленные пользователями на фильмы.
- `userId`: Уникальный идентификатор пользователя.
- `movieId`: Уникальный идентификатор фильма (ссылается на таблицу `movies`).
- `tag`: Текстовая метка, добавленная пользователем.
- `timestamp`: Временная метка, указывающая, когда была добавлена метка (обычно в формате Unix).

# Загружаем данные

In [ ]:
# Установка необходимых библиотек
!pip install surprise

In [ ]:
# Импортируем библиотеки
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans, KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.matrix_factorization import SVD

In [ ]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


# Проверяем загруженные данные

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


# Объединение данных

In [ ]:
# Объединение таблиц рейтингов и фильмов по movieId
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
# Удаление пропусков
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [ ]:
# Смотрим минимальный рейтинг, чтобы задать параметр в Readerratings.rating.min()
ratings.rating.min()



0.5

In [ ]:
# Смотрим максимальный рейтинг, чтобы задать параметр в Readerratings.rating.min()
ratings.rating.max()



5.0

# Создание DataFrame для Surprise

In [ ]:
# Формируем DataFrame для работы с пакетом surprise
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
# Указываем входные параметры для модели
reader = Reader(rating_scale=(0.5, 5.0))
# Загружаем данные в формат, подходящий для surprise
data = Dataset.load_from_df(dataset[['uid', 'iid', 'rating']], reader)

In [ ]:
def validate_model(model_init, data):
    scores = cross_validate(model_init,
                   data,
                   measures=['rmse'],
                   cv=5)['test_rmse']
    for ind in range(len(scores)):
        print(f'Fold # {ind+1}\nrmse: {scores[ind]}')

In [ ]:
# Item-based KNN:
validate_model(KNNWithMeans(sim_options={'user_based':False}, verbose=False), data)

Fold # 1
rmse: 0.90004086146604
Fold # 2
rmse: 0.8916003374262875
Fold # 3
rmse: 0.8844261569987438
Fold # 4
rmse: 0.8928167429665068
Fold # 5
rmse: 0.8912964801315911


In [ ]:
# User-based KNN_Zscore:
validate_model(KNNWithMeans(sim_options={'user_based':True}, verbose=False), data)

Fold # 1
rmse: 0.8909647215110407
Fold # 2
rmse: 0.8951505976799315
Fold # 3
rmse: 0.890338861946767
Fold # 4
rmse: 0.9043623462445883
Fold # 5
rmse: 0.9089577395293044


In [ ]:
# SVD:
validate_model(SVD(n_factors=50, n_epochs=20, lr_all=0.005, random_state=5), data)

Fold # 1
rmse: 0.8781528121646415
Fold # 2
rmse: 0.8711346782359952
Fold # 3
rmse: 0.8645984507466962
Fold # 4
rmse: 0.8724261643188114
Fold # 5
rmse: 0.8630889361750067


In [ ]:
# CoClustering:
validate_model(CoClustering(random_state=5, verbose=False), data)

Fold # 1
rmse: 0.9425716077129543
Fold # 2
rmse: 0.9564993521590895
Fold # 3
rmse: 0.9443143295372974
Fold # 4
rmse: 0.950167874093733
Fold # 5
rmse: 0.9462740082349632


# Выводы

   - KNN на основе продуктов и пользователей показал результаты, не соответствующие требованиям задания;
   - CoClustering - показал еще более низкую точность;
   - SVD - единственная модель, показавшая необходимый результат RMSE.